In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=09233207d2679fa256c1f1588a6f357a9d76cbc250276560832fafed8105dd8d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Accidents DF').getOrCreate()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [5]:
from google.colab import drive
drive.mount('/content/drive/')
df = spark.read.csv("drive/My Drive/Colab Notebooks/Road_accidents.csv",header=True,inferSchema=True)
df.show()
df.printSchema()

Mounted at /content/drive/
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12|    14|        8|     14|       6|  

In [6]:
df.groupBy("YEAR").sum("JANUARY")\
     .show()

+----+------------+
|YEAR|sum(JANUARY)|
+----+------------+
|2003|       28209|
|2007|       36432|
|2006|       33535|
|2013|       39185|
|2014|       39699|
|2004|       30722|
|2012|       38395|
|2009|       35346|
|2001|       27039|
|2005|       35740|
|2010|       36674|
|2011|       38305|
|2008|       36012|
|2002|       27426|
+----+------------+



In [7]:
from pyspark.sql.functions import sum
df1=df.select(sum(df.JANUARY).alias("JANUARY"),
          sum(df.FEBRUARY).alias("FEBRUARY"),
          sum(df.FEBRUARY).alias("MARCH"),
          sum(df.FEBRUARY).alias("APRIL"),
          sum(df.FEBRUARY).alias("MAY"),
          sum(df.FEBRUARY).alias("JUNE"),
          sum(df.FEBRUARY).alias("JULY"),
          sum(df.FEBRUARY).alias("AUGUST"),
          sum(df.FEBRUARY).alias("SEPTEMBER"),
          sum(df.FEBRUARY).alias("OCTOBER"),
          sum(df.FEBRUARY).alias("NOVEMBER"),
          sum(df.FEBRUARY).alias("DECEMBER")


    ).show()

+-------+--------+------+------+------+------+------+------+---------+-------+--------+--------+
|JANUARY|FEBRUARY| MARCH| APRIL|   MAY|  JUNE|  JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|
+-------+--------+------+------+------+------+------+------+---------+-------+--------+--------+
| 482719|  459272|459272|459272|459272|459272|459272|459272|   459272| 459272|  459272|  459272|
+-------+--------+------+------+------+------+------+------+---------+-------+--------+--------+



In [11]:
#Add column 'Total', giving sum of accidents in each month
total = df.withColumn('TOTAL',(col('JANUARY')+col('FEBRUARY')+col('MARCH')+col('APRIL')+col('MAY')\
    +col('JUNE')+col('JULY')+col('AUGUST')+col('SEPTEMBER')+col('OCTOBER')+col('NOVEMBER')+col('DECEMBER')))
total.show()


+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-----+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|TOTAL|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-----+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|  181|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|  168|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|  180|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|  215|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|  206|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12|    14|        

In [12]:
#Which state has highest number of accidents in year 2013?
high_acc = total.select('STATE/UT','YEAR','TOTAL').filter(col('YEAR') == 2013)
high_acc.where(col('TOTAL') == high_acc.agg({'TOTAL':'max'}).collect()[0][0]).show()

+----------+----+-----+
|  STATE/UT|YEAR|TOTAL|
+----------+----+-----+
|Tamil Nadu|2013|66238|
+----------+----+-----+



In [13]:
#Find the average monthly accidents for each state.
monthAcc = df.withColumn('AVG',(col('JANUARY')+col('FEBRUARY')+col('MARCH')+col('APRIL')+col('MAY')+col('JUNE')+col('JULY')+col('AUGUST')
    +col('SEPTEMBER')+col('OCTOBER')+col('NOVEMBER')+col('DECEMBER'))/12)
monthAcc.select('STATE/UT','AVG').groupBy('STATE/UT','AVG').count().distinct().show(30)

+----------------+------------------+-----+
|        STATE/UT|               AVG|count|
+----------------+------------------+-----+
|       Rajasthan|           1714.25|    1|
|  Madhya Pradesh|           2547.75|    1|
|         Manipur|44.833333333333336|    1|
|           Assam| 304.6666666666667|    1|
|     West Bengal|1205.6666666666667|    1|
|      Tamil Nadu|            4331.5|    1|
|       Karnataka|3765.8333333333335|    1|
|  Madhya Pradesh|2836.1666666666665|    1|
|        Nagaland|               3.5|    1|
|         Tripura| 68.16666666666667|    1|
|     West Bengal|1090.4166666666667|    1|
|    D & N Haveli| 7.583333333333333|    1|
|         Haryana|            872.75|    1|
|       Karnataka|3851.6666666666665|    1|
|         Haryana|            952.25|    1|
|          Kerala|            3014.5|    1|
|      Puducherry|120.91666666666667|    1|
|       Rajasthan|2025.1666666666667|    1|
|   A & N Islands|17.166666666666668|    1|
|       Rajasthan|1666.583333333

In [14]:

#Which month has highest accidents?
highest_acc = df.select([col for col in df.columns if col!= 'STATE/UT' and col!='YEAR'])
var  = {}
for i in highest_acc.columns:
  var[i] = highest_acc.agg({i:'max'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()


+-----+-----+
|month|value|
+-----+-----+
|  MAY| 9330|
+-----+-----+



In [15]:
#Which state has more accidents from year 2001 to 2014?
more_acc = total.select('TOTAL','YEAR','STATE/UT').where(col('YEAR') <= 2014)
more_accident = more_acc.groupBy('STATE/UT','TOTAL').count().agg({'TOTAL':'sum'}).collect()[0][0]
more_accident

5599501

In [17]:
#Which month has more accidents over the year?
highest_acc = df.select([col for col in df.columns if col!= 'STATE/UT' and col!='YEAR'])
var  = {}
for i in highest_acc.columns:
  var[i] = highest_acc.agg({i:'max'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()

+-----+-----+
|month|value|
+-----+-----+
|  MAY| 9330|
+-----+-----+

